In [1]:
import copy

import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


from keras.layers import Input, Dense, Flatten,Activation
from keras.layers import Conv1D, MaxPooling1D,Dropout,LSTM
from keras.layers import Embedding
from keras.models import Model

from numpy import zeros
from numpy import asarray

from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential

from keras.layers import Dense
from keras.layers import Flatten

from keras.layers.embeddings import Embedding
import tensorflow

# from gensim.models import FastText

In [2]:
tensorflow.__version__

'2.4.1'

In [3]:
tensorflow.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12736042683052316238,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 23354508672
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13921353494587940759
 physical_device_desc: "device: 0, name: TITAN RTX, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [5]:
new_sentences = list()

training_file = '../data/train.tsv'

for i, line in enumerate(open(training_file).readlines()):
    row = line.strip().split('\t')
    if len(row) > 1:
        tokens = row[1].strip()

        tokens = 'BEGIN' + tokens  + 'SPACE2'

        tokens = tokens.replace(' ', ' SPACE ')

        tokens = tokens.replace('>', ' ')

        tokens = tokens.replace('BEGIN', 'BEGIN ')
        tokens = tokens.replace('SPACE2', ' SPACE')

        new_sentences.append(tokens)

In [6]:
new_sentences[:1]

['BEGIN а маравкэва ра тэн SPACE таа ’ко йӈы н SPACE']

In [7]:
for i in range(len(new_sentences)):
    new_sentences[i] = new_sentences[i].replace('  ', ' ')
    new_sentences[i] = new_sentences[i].split(' ')

In [8]:
new_sentences[:1]

[['BEGIN',
  'а',
  'маравкэва',
  'ра',
  'тэн',
  'SPACE',
  'таа',
  '’ко',
  'йӈы',
  'н',
  'SPACE']]

In [9]:
max_len = 0
for sent in new_sentences:
    if len(sent) > max_len:
        max_len = len(sent)

In [10]:
max_len

131

In [11]:
generated_sentences = list()
target_of_generated_sentences = list()
for i, sent in enumerate(new_sentences):
    for j in range(len(sent)-1):
        if j == 0:
            generated_sentences.append(sent[j])
        else:
            generated_sentences.append(sent[:j+1])
        target_of_generated_sentences.append(sent[j+1])


In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(generated_sentences)

vocab_size = len(tokenizer.word_index) + 1
word_index = tokenizer.word_index

In [13]:
generated_sentences[1]

['BEGIN', 'а']

In [14]:
encoded_docs = tokenizer.texts_to_sequences(generated_sentences)

In [15]:
encoded_docs[1]

[2, 15]

In [16]:
padded_docs = pad_sequences(encoded_docs, maxlen=max_len, padding='post')

In [17]:
padded_docs[1]

array([ 2, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [18]:
len(padded_docs)

448152

In [19]:
le = LabelEncoder()
target_of_generated_sentences = le.fit_transform(target_of_generated_sentences)

In [20]:
target_of_generated_sentences

array([   4, 5123, 7883, ..., 6095, 5982,    0])

In [21]:
different_labels = len(pd.unique(target_of_generated_sentences))

In [22]:
different_labels

17405

In [23]:
#with tensorflow.device('/device:GPU:0'):
model = Sequential()
embedding_layer = Embedding(vocab_size, 50, input_length=max_len)
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(1024, 10, activation='relu')(embedded_sequences)
x = MaxPooling1D(2)(x)
x = Conv1D(512, 5, activation='relu')(x)
x = MaxPooling1D(2)(x)
x = Conv1D(256, 3, activation='relu')(x)
x = MaxPooling1D(4)(x)
x = Flatten()(x)
x = Dense(different_labels*2, activation='relu')(x)
preds = Dense(different_labels, activation='softmax')(x)

model = Model(sequence_input, preds)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

model.fit(padded_docs, target_of_generated_sentences, epochs=500, verbose=1, batch_size=2048)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 131)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 131, 50)           870350    
_________________________________________________________________
conv1d (Conv1D)              (None, 122, 1024)         513024    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 61, 1024)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 57, 512)           2621952   
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 28, 512)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 26, 256)           393472

219/219 [==============================] - 205s 937ms/step - loss: 0.7667 - accuracy: 0.8499
Epoch 62/500
219/219 [==============================] - 205s 937ms/step - loss: 0.7598 - accuracy: 0.8518
Epoch 63/500
219/219 [==============================] - 205s 937ms/step - loss: 0.7583 - accuracy: 0.8521
Epoch 64/500
219/219 [==============================] - 205s 937ms/step - loss: 0.7514 - accuracy: 0.8535
Epoch 65/500
219/219 [==============================] - 205s 936ms/step - loss: 0.7501 - accuracy: 0.8531
Epoch 66/500
219/219 [==============================] - 205s 936ms/step - loss: 0.7577 - accuracy: 0.8522
Epoch 67/500
219/219 [==============================] - 205s 935ms/step - loss: 0.7551 - accuracy: 0.8523
Epoch 68/500
219/219 [==============================] - 205s 935ms/step - loss: 0.7391 - accuracy: 0.8555
Epoch 69/500
219/219 [==============================] - 205s 935ms/step - loss: 0.7507 - accuracy: 0.8525
Epoch 70/500
219/219 [==============================] - 205

KeyboardInterrupt: 

In [45]:
model.save_weights('cnn_tokens.h5')

In [37]:
# lets create morph parser from all data
morph = dict()

training_file = '../data/train.tsv'

for i, line in enumerate(open(training_file).readlines()):
    row = line.strip().split('\t')
    if len(row) > 1:
        tokens = row[0].split(' ')
        tokens_with_morph = row[1].split(' ')
        for i in range(len(tokens)-1):
            morph[tokens[i]] = tokens_with_morph[i]
            

training_file = '../data/dev.tsv'

for i, line in enumerate(open(training_file).readlines()):
    row = line.strip().split('\t')
    if len(row) > 1:
        tokens = row[0].split(' ')
        tokens_with_morph = row[1].split(' ')
        for i in range(len(tokens)-1):
            morph[tokens[i]] = tokens_with_morph[i]


training_file = '../data/test/test.tsv'

for i, line in enumerate(open(training_file).readlines()):
    row = line.strip().split('\t')
    if len(row) > 1:
        tokens = row[0].split(' ')
        tokens_with_morph = row[1].split(' ')
        for i in range(len(tokens)-1):
            morph[tokens[i]] = tokens_with_morph[i]

In [38]:
len(morph)

30284

In [27]:
exist_text = ['BEGIN']
exist_text = pad_sequences(tokenizer.texts_to_sequences([exist_text]), maxlen=max_len, padding='post')

In [28]:
next_token = model.predict(exist_text)

In [29]:
next_token = np.argmax(next_token)
next_token = le.inverse_transform([next_token])

In [30]:
next_token

array(['ны'], dtype='<U21')

In [31]:
next_token = ''
exist_text = ['BEGIN', 'вынэ']

#tokenizer.texts_to_sequences(exist_text)
cnt = 0
while cnt < 10:
    exist_text_array = pad_sequences(tokenizer.texts_to_sequences([exist_text]), maxlen=max_len, padding='post')
    next_token = model.predict(exist_text_array)
    next_token = np.argmax(next_token)
    next_token = le.inverse_transform([next_token])
    next_token = next_token[0]
    exist_text += [next_token]
    cnt += 1

In [32]:
def predict_next_word(exist_text):
    first_flag = False
    if exist_text[0] == '#':
        exist_text = ['BEGIN']
        first_flag = True
    else:
        exist_text_save = copy.copy(exist_text)
        exist_text = ['BEGIN']
        for tok in exist_text_save:
            morph_tokens = morph[tok].split('>')
            for mrt in morph_tokens:
                exist_text += [mrt]
            exist_text += ['SPACE']
    
    next_token = ''
    while next_token != 'SPACE':
        exist_text_array = pad_sequences(tokenizer.texts_to_sequences([exist_text]), maxlen=max_len, padding='post')
        next_token = model.predict(exist_text_array)
        next_token = np.argmax(next_token)
        next_token = le.inverse_transform([next_token])
        next_token = next_token[0]
        exist_text += [next_token]
    
    if first_flag:
        index_1 = np.where(np.array(exist_text) == 'BEGIN')[0][-1]
        index_2 = np.where(np.array(exist_text) == 'SPACE')[0][-1]
    else:
        index_1 = np.where(np.array(exist_text) == 'SPACE')[0][-2]
        index_2 = np.where(np.array(exist_text) == 'SPACE')[0][-1]

    next_word = ''.join(exist_text[index_1+1:index_2])
    
    return next_word

In [ ]:
f = open('output_copy.tsv', 'w')

n_tokens = 0
hits = 0

for line in tqdm(open('../data/dev.tsv').readlines()):
    # Split into two columns
    row = line.strip().split('\t')
    # Our tokens are in column one, split by space
    tokens = row[0].split(' ')
    # The test tokens are the beginning of sentence symbol + the list of tokens
    tst_tokens = ['#'] + tokens
    # Increment the number of tokens by the length of the list containing the tokens
    n_tokens += len(tokens)

    # This is our output
    output = []

    # For each of the tokens in the "tst_tokens" list (e.g. the list + the beginning of sentence symbol)
    for i in range(len(tst_tokens)-1):
        first = tst_tokens[i]  # First token in bigram
        second = tst_tokens[i+1]  # Second token in bigram

        if i == 0:
            predicted_second = predict_next_word(tst_tokens[i])
        else:
            predicted_second = predict_next_word(tst_tokens[1:i+1])
            
        

        if predicted_second == second:
            # We add this whole token to the output
            # e.g. a single click on a prediction

            output.append(predicted_second)
            #print(predicted_second)
            #print(output)
            # Increment the number of hits by 1
            hits += 1
        else:
            # Otherwise we add each individual character to the output
            # e.g. writing out each of the individual clicks
            output += [c for c in second]

        output.append('_')

    
    print('%s\t%s' % (row[0], ' '.join(output)), file=f)

f.close()

In [34]:
print(hits)

37


In [35]:
!python3 ../evaluate.py ../data/dev.tsv ./output_copy.tsv

Characters: 37897
Tokens: 8788
Clicks: 37752
Clicks/Token: 4.295857988165681
Clicks/Character: 0.9961738396179117


In [39]:
f = open('output_copy_test.tsv', 'w')

n_tokens = 0
hits = 0

for line in tqdm(open('../data/test/test.tsv').readlines()):
    # Split into two columns
    row = line.strip().split('\t')
    # Our tokens are in column one, split by space
    tokens = row[0].split(' ')
    # The test tokens are the beginning of sentence symbol + the list of tokens
    tst_tokens = ['#'] + tokens
    # Increment the number of tokens by the length of the list containing the tokens
    n_tokens += len(tokens)

    # This is our output
    output = []

    # For each of the tokens in the "tst_tokens" list (e.g. the list + the beginning of sentence symbol)
    for i in range(len(tst_tokens)-1):
        first = tst_tokens[i]  # First token in bigram
        second = tst_tokens[i+1]  # Second token in bigram

        if i == 0:
            predicted_second = predict_next_word(tst_tokens[i])
        else:
            predicted_second = predict_next_word(tst_tokens[1:i+1])
            
        

        if predicted_second == second:
            # We add this whole token to the output
            # e.g. a single click on a prediction

            output.append(predicted_second)
            #print(predicted_second)
            #print(output)
            # Increment the number of hits by 1
            hits += 1
        else:
            # Otherwise we add each individual character to the output
            # e.g. writing out each of the individual clicks
            output += [c for c in second]

        output.append('_')

    
    print('%s\t%s' % (row[0], ' '.join(output)), file=f)

f.close()

100%|██████████| 1006/1006 [07:30<00:00,  2.24it/s]


In [41]:
hits

6

In [43]:
!python3 ../evaluate.py ../data/test/test.tsv ./output_copy_test.tsv

Characters: 37927
Tokens: 8374
Clicks: 37911
Clicks/Token: 4.527227131597803
Clicks/Character: 0.9995781369472935
